In [1]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def fetch_table(database, table, query = ""):
    try:
        user = "postgres"
        password = "password"
        host = "localhost"
        port = 5432

        connection_string = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}'
        engine = create_engine(connection_string)

        if query == "":
            query = f"SELECT * FROM {table};"

        df = pd.read_sql(query, engine)

        return df
    
    except Exception as error:
        print(f"Error fetching data from table '{table}' in database '{database}': {error}")
        return None

In [3]:
# merging route weather table

# "SELECT t1.route_id, t1."Date", t1.temp, t1.wind_speed, t1.precip, t1.humidity, t1.visibility, t1.pressure, t1.condition, t1.intensity,
#        t2.truck_id, t2.route_id, t2.departure_date, t2.estimated_arrival, t2.delay
# FROM route_weather_table t1
# JOIN trucks_schedule_table t2
# ON t1.route_id = t2.route_id
# WHERE t1."Date" BETWEEN t2.departure_date AND t2.estimated_arrival;" 

In [4]:
route_df = fetch_table("td_clean_data", "route_table")
route_weather_df = fetch_table("td_clean_data", "route_weather_table")
traffic_table_df = fetch_table("td_clean_data", "traffic_table")
trucks_df = fetch_table("td_clean_data", "trucks_table")
drivers_df = fetch_table("td_clean_data", "drivers_table")
trucks_schedule_df = fetch_table("td_clean_data", "trucks_schedule_table")

In [5]:
# Function to round to nearest 6-hour interval
def round_to_nearest_6hr(dt):
    hour = dt.hour
    rounded_hour = int(round(hour / 6) * 6)
    if rounded_hour == 24:
        dt = dt + pd.Timedelta(days=1)
        rounded_hour = 0
    return dt.replace(hour=rounded_hour, minute=0, second=0)

def round_to_nearest_hour(dt):
    minute = dt.minute
    if minute >= 30:
        dt = dt + pd.Timedelta(hours=1)
    return dt.replace(minute=0, second=0)

In [6]:
import pandas as pd

route_weather_df['Date'] = pd.to_datetime(route_weather_df['Date'])
trucks_schedule_df['departure_date'] = pd.to_datetime(trucks_schedule_df['departure_date'])
trucks_schedule_df['estimated_arrival'] = pd.to_datetime(trucks_schedule_df['estimated_arrival'])

# Merge the two DataFrames on 'route_id'
merged_df = pd.merge(route_weather_df, trucks_schedule_df, on='route_id')

# to handle trips that last less than 6hrs
merged_df['departure_date_6hrs'] = merged_df['departure_date'].apply(round_to_nearest_6hr)
merged_df['estimated_arrival_6hrs'] = merged_df['estimated_arrival'].apply(round_to_nearest_6hr)

# Filter the rows where 'Date' falls between 'departure_date' and 'estimated_arrival'
filtered_df = merged_df[(merged_df['Date'] >= merged_df['departure_date_6hrs']) & (merged_df['Date'] <= merged_df['estimated_arrival_6hrs'])]


In [7]:
len(filtered_df)

64066

In [8]:
trucks_schedule_df.drop_duplicates()

,truck_id,route_id,departure_date,estimated_arrival,delay
0,30312694,R-b236e347,2019-01-01 07:00:00,2019-01-01 13:13:12.000000000,0
1,59856374,R-29ea762e,2019-01-01 07:00:00,2019-01-02 04:01:12.000000000,0
2,12602955,R-a3d67783,2019-01-01 07:00:00,2019-01-01 07:45:36.000000000,0
3,46619422,R-31ec9310,2019-01-01 07:00:00,2019-01-01 20:46:48.000000000,0
4,10140178,R-a07c5dbd,2019-01-01 07:00:00,2019-01-01 21:34:11.999999999,0
...,...,...,...,...,...
12303,31047945,R-1484a7ea,2019-02-06 07:00:00,2019-02-13 22:40:12.000000000,0
12304,14758432,R-927cf900,2019-02-06 07:00:00,2019-02-13 18:52:48.000000000,1
12305,31370619,R-5a83ad98,2019-02-06 07:00:00,2019-02-14 08:01:48.000000000,1
12306,67332883,R-991530bc,2019-02-06 07:00:00,2019-02-13 11:14:24.000000000,1


In [9]:
aggregated_df = filtered_df.groupby(['truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay']).agg({
    'temp': 'mean',
    'wind_speed': 'mean',
    'precip': 'mean',
    'humidity': 'mean',
    'visibility': 'mean',
    'pressure': 'mean'
}).reset_index()

In [10]:
# Merged df with weather
len(aggregated_df)

12308

In [11]:
# Merging with routes table
truck_delay_classification_df = pd.merge(aggregated_df, route_df, on='route_id')
truck_delay_classification_df.head()

,truck_id,route_id,departure_date,estimated_arrival,delay,temp,wind_speed,precip,humidity,visibility,pressure,origin_id,destination_id,distance,average_hours,velocity
0,10008392,R-06dfe39e,2019-02-02 07:00:00,2019-02-04 18:55:12,1,49.000000,5.818182,0.0,80.909091,5.818182,1015.181818,C-731988ba,C-03bb3e48,2995.85,59.92,49.997497
1,10008392,R-35dca729,2019-01-01 07:00:00,2019-01-03 09:15:36,1,56.600000,9.400000,0.0,72.200000,5.700000,1013.300000,C-73ae5412,C-b5ff31cd,2512.99,50.26,49.999801
2,10008392,R-51d1a32a,2019-01-17 07:00:00,2019-01-19 17:57:36,1,46.636364,8.909091,0.0,76.181818,6.000000,1016.818182,C-ef47bdcd,C-731988ba,2947.83,58.96,49.997117
3,10008392,R-7ab5ae85,2019-01-09 07:00:00,2019-01-10 22:22:12,0,58.750000,7.000000,0.0,82.250000,4.000000,1016.000000,C-3dbd8b2e,C-90e30162,1968.52,39.37,50.000508
4,10008392,R-c6f8e036,2019-02-10 07:00:00,2019-02-12 18:55:12,1,48.181818,6.545455,0.0,87.727273,5.545455,1011.727273,C-03bb3e48,C-731988ba,2995.85,59.92,49.997497


In [12]:
len(trucks_schedule_df)

12308

In [13]:
truck_delay_classification_df.head()

,truck_id,route_id,departure_date,estimated_arrival,delay,temp,wind_speed,precip,humidity,visibility,pressure,origin_id,destination_id,distance,average_hours,velocity
0,10008392,R-06dfe39e,2019-02-02 07:00:00,2019-02-04 18:55:12,1,49.000000,5.818182,0.0,80.909091,5.818182,1015.181818,C-731988ba,C-03bb3e48,2995.85,59.92,49.997497
1,10008392,R-35dca729,2019-01-01 07:00:00,2019-01-03 09:15:36,1,56.600000,9.400000,0.0,72.200000,5.700000,1013.300000,C-73ae5412,C-b5ff31cd,2512.99,50.26,49.999801
2,10008392,R-51d1a32a,2019-01-17 07:00:00,2019-01-19 17:57:36,1,46.636364,8.909091,0.0,76.181818,6.000000,1016.818182,C-ef47bdcd,C-731988ba,2947.83,58.96,49.997117
3,10008392,R-7ab5ae85,2019-01-09 07:00:00,2019-01-10 22:22:12,0,58.750000,7.000000,0.0,82.250000,4.000000,1016.000000,C-3dbd8b2e,C-90e30162,1968.52,39.37,50.000508
4,10008392,R-c6f8e036,2019-02-10 07:00:00,2019-02-12 18:55:12,1,48.181818,6.545455,0.0,87.727273,5.545455,1011.727273,C-03bb3e48,C-731988ba,2995.85,59.92,49.997497


In [14]:
import pandas as pd

# Assuming you have the two DataFrames: truck_schedule_df and aggregated_df

# Step 1: Get all column names from truck_schedule_df
merge_columns = list(trucks_schedule_df.columns)

# Step 2: Perform a left merge on all columns from truck_schedule_df
comparison_df = pd.merge(trucks_schedule_df, aggregated_df, on=merge_columns, how='left', indicator=True)

# Step 3: Filter rows where the row is only in truck_schedule_df (i.e., not in the aggregated_df)
missing_rows_df = comparison_df[comparison_df['_merge'] == 'left_only']

# Drop the '_merge' column as it is not needed anymore
missing_rows_df = missing_rows_df.drop(columns=['_merge'])

# Display the rows from truck_schedule_df that are not in aggregated_df
missing_rows_df


,truck_id,route_id,departure_date,estimated_arrival,delay,temp,wind_speed,precip,humidity,visibility,pressure


In [15]:
null_values = traffic_table_df.isnull().sum()
traffic_table_df.fillna(0)
# Display the result
print(null_values)

route_id             0
date                 0
hour                 0
no_of_vehicles    1152
accident             0
dtype: int64


In [16]:
# merging with traffic table
traffic_table_df['datetime'] = pd.to_datetime(traffic_table_df['date']) + pd.to_timedelta(traffic_table_df['hour'], unit='h')

trucks_schedule_df['departure_date'] = pd.to_datetime(trucks_schedule_df['departure_date'])
trucks_schedule_df['estimated_arrival'] = pd.to_datetime(trucks_schedule_df['estimated_arrival'])

trucks_schedule_df['departure_date_1hr'] = trucks_schedule_df['departure_date'].apply(round_to_nearest_hour)
trucks_schedule_df['estimated_arrival_1hr'] = trucks_schedule_df['estimated_arrival'].apply(round_to_nearest_hour)


merged_df = pd.merge(traffic_table_df, trucks_schedule_df, on='route_id')

filtered_df = merged_df[(merged_df['datetime'] >= merged_df['departure_date_1hr']) & 
                        (merged_df['datetime'] <= merged_df['estimated_arrival_1hr'])]

filtered_df.head()

,route_id,date,hour,no_of_vehicles,accident,datetime,truck_id,departure_date,estimated_arrival,delay,departure_date_1hr,estimated_arrival_1hr
27,R-ada2a391,2019-01-01,500,564.0,0,2019-01-21 20:00:00,11316377,2019-01-21 07:00:00,2019-01-22 17:42:00,1,2019-01-21 07:00:00,2019-01-22 18:00:00
54,R-ada2a391,2019-01-01,1000,2925.0,0,2019-02-11 16:00:00,28295338,2019-02-10 07:00:00,2019-02-11 17:42:00,1,2019-02-10 07:00:00,2019-02-11 18:00:00
120,R-ada2a391,2019-01-02,0,714.0,0,2019-01-02 00:00:00,29596705,2019-01-01 07:00:00,2019-01-02 17:42:00,0,2019-01-01 07:00:00,2019-01-02 18:00:00
251,R-ada2a391,2019-01-03,200,645.0,0,2019-01-11 08:00:00,72146642,2019-01-11 07:00:00,2019-01-12 17:42:00,1,2019-01-11 07:00:00,2019-01-12 18:00:00
278,R-ada2a391,2019-01-03,700,2065.0,0,2019-02-01 04:00:00,29178779,2019-01-31 07:00:00,2019-02-01 17:42:00,1,2019-01-31 07:00:00,2019-02-01 18:00:00


In [17]:
aggregated_df = filtered_df.groupby(['truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay']).agg({
    'no_of_vehicles': 'mean',
    'accident': 'sum'
}).reset_index()

In [18]:
aggregated_df

,truck_id,route_id,departure_date,estimated_arrival,delay,no_of_vehicles,accident
0,10008392,R-06dfe39e,2019-02-02 07:00:00,2019-02-04 18:55:12,1,897.904762,0
1,10008392,R-35dca729,2019-01-01 07:00:00,2019-01-03 09:15:36,1,509.000000,0
2,10008392,R-51d1a32a,2019-01-17 07:00:00,2019-01-19 17:57:36,1,531.153846,1
3,10008392,R-7ab5ae85,2019-01-09 07:00:00,2019-01-10 22:22:12,0,533.000000,0
4,10008392,R-c6f8e036,2019-02-10 07:00:00,2019-02-12 18:55:12,1,1147.814815,3
...,...,...,...,...,...,...,...
10930,99981667,R-92f26285,2019-02-06 07:00:00,2019-02-06 12:15:00,1,1031.333333,0
10931,99981667,R-abbf83d1,2019-01-16 07:00:00,2019-01-16 22:05:24,1,617.000000,0
10932,99981667,R-e9bf94ff,2019-01-01 07:00:00,2019-01-02 00:16:12,0,492.000000,0
10933,99981667,R-f363ad37,2019-02-09 07:00:00,2019-02-10 02:32:24,1,778.125000,0


In [59]:
trucks_schedule_df.columns

Index(['truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay',
       'departure_date_1hr', 'estimated_arrival_1hr'],
      dtype='object')

In [19]:
import pandas as pd

# Assuming you have the two DataFrames: truck_schedule_df and aggregated_df

# Step 1: Get all column names from truck_schedule_df
merge_columns = ['truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay']

# Step 2: Perform a left merge on all columns from truck_schedule_df
comparison_df = pd.merge(trucks_schedule_df, aggregated_df, on=merge_columns, how='left', indicator=True)

# Step 3: Filter rows where the row is only in truck_schedule_df (i.e., not in the aggregated_df)
missing_rows_df = comparison_df[comparison_df['_merge'] == 'left_only']

# Drop the '_merge' column as it is not needed anymore
missing_rows_df = missing_rows_df.drop(columns=['_merge'])

# Display the rows from truck_schedule_df that are not in aggregated_df
missing_rows_df

,truck_id,route_id,departure_date,estimated_arrival,delay,departure_date_1hr,estimated_arrival_1hr,no_of_vehicles,accident
0,30312694,R-b236e347,2019-01-01 07:00:00,2019-01-01 13:13:12.000000000,0,2019-01-01 07:00:00,2019-01-01 13:00:00.000000000,NaN,NaN
2,12602955,R-a3d67783,2019-01-01 07:00:00,2019-01-01 07:45:36.000000000,0,2019-01-01 07:00:00,2019-01-01 08:00:00.000000000,NaN,NaN
3,46619422,R-31ec9310,2019-01-01 07:00:00,2019-01-01 20:46:48.000000000,0,2019-01-01 07:00:00,2019-01-01 21:00:00.000000000,NaN,NaN
4,10140178,R-a07c5dbd,2019-01-01 07:00:00,2019-01-01 21:34:11.999999999,0,2019-01-01 07:00:00,2019-01-01 22:00:00.999999999,NaN,NaN
5,22653688,R-af4b19be,2019-01-01 07:00:00,2019-01-01 15:39:36.000000000,0,2019-01-01 07:00:00,2019-01-01 16:00:00.000000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...
8481,20958889,R-9b13723a,2019-02-09 07:00:00,2019-02-09 07:27:36.000000000,0,2019-02-09 07:00:00,2019-02-09 07:00:00.000000000,NaN,NaN
9045,21071765,R-d73c56ee,2019-02-12 07:00:00,2019-02-12 07:27:36.000000000,1,2019-02-12 07:00:00,2019-02-12 07:00:00.000000000,NaN,NaN
9098,29054232,R-1b7d2dcc,2019-02-12 07:00:00,2019-02-12 07:04:48.000000000,0,2019-02-12 07:00:00,2019-02-12 07:00:00.000000000,NaN,NaN
9180,21162393,R-564643b5,2019-02-12 07:00:00,2019-02-12 07:25:48.000000000,0,2019-02-12 07:00:00,2019-02-12 07:00:00.000000000,NaN,NaN


In [72]:
trucks_schedule_df[(trucks_schedule_df["route_id"] == "R-b236e347") & (trucks_schedule_df["truck_id"] == 30312694)]

,truck_id,route_id,departure_date,estimated_arrival,delay,departure_date_1hr,estimated_arrival_1hr
0,30312694,R-b236e347,2019-01-01 07:00:00,2019-01-01 13:13:12,0,2019-01-01 07:00:00,2019-01-01 13:00:00


In [74]:
test = merged_df[(merged_df["route_id"] == "R-b236e347") & (merged_df["truck_id"] == 30312694)]

filtered_df = test[(test['datetime'] >= test['departure_date_1hr']) & 
                         (test['datetime'] <= test['estimated_arrival_1hr'])]

In [80]:
filtered_test = test[test['datetime'].dt.date == pd.to_datetime('2019-01-01').date()]
  
test[test['datetime'].dt.date == pd.to_datetime('2019-01-01').date()][['datetime', 'departure_date_1hr', 'estimated_arrival_1hr']].sort_values(by='datetime', ascending=True)

,datetime,departure_date_1hr,estimated_arrival_1hr
18768,2019-01-01,2019-01-01 07:00:00,2019-01-01 13:00:00
